In [ ]:
!pip install stable-baselines3[extra]

In [4]:
import numpy as np
import gym
from gym import spaces

class TicTacToeEnv(gym.Env):
  """
  Класс TicTacToeEnv представляет собой среду для игры в крестики-нолики.
  Библиотека gym предоставляет интерфейс для создания и управлениz средами обучения.
  """

  metadata = {'render.moves': ['human']}

  def __init__(self):
    """
    Инициализация среды, определение пространства действий и состояний.
    """
    super(TicTacToeEnv, self).__init__()
    self.action_space = spaces.Discrete(9)
    self.observation_space = spaces.Box(low=0, high=2, shape=(3, 3), dtype=np.int)
    self.state = np.zeros((3, 3), dtype=np.int)
    self.current_player = 1

  def step(self, action):
    """
    Выполнение хода в игре. Принимает действие, обновляет состояние игры,
    возвращает новое состояние, награду, флаг завершения игры.
    """
    if self.is_valid_action(action):
      row, col = divmod(action, 3)
      self.state[row, col] = self.current_player
      winner = self.get_winner()
      done = winner is not None or np.all(self.state != 0)
      reward = 1 if winner == self.current_player else 0
      self.switch_player()
      return self.state, reward, done, {}
    else:
      return self.state, -1, True, {}

  def reset(self):
    """
    Сброс состояния среды к начальному
    """
    self.state = np.zeros((3, 3), dtype=np.int)
    self.current_player = 1
    return self.state

  def render(self, mode='human', close=False):
    """
    Отображение текущего состояния игрового поля.
    """
    if close:
      return
    for row in self.state:
      print(" ".join('XO'[i-1] if i > 0 else '.' for i in row))
    print("")

  def close(self):
    """
    Закрытие среды.
    """
    pass

  def switch_player(self):
    """
    Смена текущего игрока.
    """
    self.current_player = 3 - self.current_player

  def is_valid_action(self, action):
    """
    Проверка, является ли действие допустимым (можно ли сделать ход в выбранную ячейку).
    """
    row, col = divmod(action, 3)
    return self.state[row, col] == 0

  def is_game_over(self):
    """
    Проверка, закончилась ли игра (есть победитель или все ячейки заполнены).
    """
    return self.get_winner() is not None or np.all(self.state != 0)

  def get_winner(self):
    """
    Проверка строк, столбцов и диагоналей на наличие победителя.
    """
    for player in [1, 2]:
      if np.any(np.all(self.state == player, axis=0)) or \
        np.any(np.all(self.state == player, axis=1)) or \
        np.all(np.diag(self.state) == player) or \
        np.all(np.diag(np.fliplr(self.state)) == player):
        return player
    return None

In [11]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

vec_env = make_vec_env(lambda: TicTacToeEnv(), n_envs=4)

# Используем моедль PPO
model = PPO("MlpPolicy", vec_env, verbose=1)

# Оценим начальную производительность агента (до обучения)
mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=10)
print(f"Средняя награда до обучения: {mean_reward:.2f} +/- {std_reward:.2f}")

# Обучим модель
model.learn(total_timesteps=125000)

# Оценим производительность агента после обучения
mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=10)
print(f"Средняя награда после обучения: {mean_reward:.2f} +/- {std_reward:.2f}")

model.save("ppo_tictactoe")

<ipython-input-4-498ce4cfacea>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.observation_space = spaces.Box(low=0, high=2, shape=(3, 3), dtype=np.int)


Using cpu device
Средняя награда до обучения: -1.00 +/- 0.00
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.61     |
|    ep_rew_mean     | -0.92    |
| time/              |          |
|    fps             | 2931     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4.7         |
|    ep_rew_mean          | -0.86       |
| time/                   |             |
|    fps                  | 1286        |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.020358715 |
|    clip_fraction        | 0.163       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.19       |
|    explai

In [12]:
# Случайный агент
def random_agent(state):
  """
  Агент, который выбирает случайный допустимый ход.
  :param state: текущее состояние игрового поля
  :return: индекс действия
  """
  available_actions = [i for i in range(9) if state[i // 3, i % 3] == 0]
  return np.random.choice(available_actions) if available_actions else None

# Игра со случайным агентом
def play_games(model, env, num_games):
    wins = 0
    losses = 0
    draws = 0

    for _ in range(num_games):
        obs = env.reset()
        done = False
        while not done:
            action, _ = model.predict(obs)
            obs, reward, done, info = env.step(action)
            if done:
                if reward == 1:
                    wins += 1
                elif reward == -1:
                    losses += 1
                else:
                    draws += 1
            else:
                # Если игра не закончилась, случайный агент делает ход
                action = random_agent(obs)
                obs, reward, done, info = env.step(action)

    print(f"Агент выиграл {wins} игр, проиграл {losses} игр, сыграл вничью {draws} игр.")

play_games(model, TicTacToeEnv(), num_games=500)


<ipython-input-4-498ce4cfacea>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.observation_space = spaces.Box(low=0, high=2, shape=(3, 3), dtype=np.int)


Агент выиграл 324 игр, проиграл 4 игр, сыграл вничью 41 игр.
